In [1]:
import os

# show the current directory

In [2]:
%pwd

'/home/fintechsys/MLflow projects/kidney-Disease-classification-Deep-Learning--project/research'

# go back to the father directory

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/fintechsys/MLflow projects/kidney-Disease-classification-Deep-Learning--project'

# every thing in the config.yamal at data_ingestion 

In [1]:
from dataclasses import dataclass
from pathlib import Path

# using @dataclass so the class will be access as variable from other file
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# to update components stage

In [4]:
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



# Creat the pipline

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-04 15:00:25,734: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-04 15:00:25,737: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-04 15:00:25,739: INFO: common: created directory at: artifacts]
[2024-02-04 15:00:25,740: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-04 15:00:25,740: INFO: 3172177572: Downloading data from https://drive.google.com/file/d/17iriTzXvzVbMtVrI1mNW0uZAwgqM9tE3/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=17iriTzXvzVbMtVrI1mNW0uZAwgqM9tE3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=17iriTzXvzVbMtVrI1mNW0uZAwgqM9tE3&confirm=t&uuid=f9e0501a-493d-4474-9db7-a8c18751f3c6
To: /home/fintechsys/MLflow projects/kidney-Disease-classification-Deep-Learning--project/artifacts/data_ingestion/data.zip
100%|██████████| 136M/136M [01:57<00:00, 1.15MB/s] 

[2024-02-04 15:02:25,464: INFO: 3172177572: Downloaded data from https://drive.google.com/file/d/17iriTzXvzVbMtVrI1mNW0uZAwgqM9tE3/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
